<font size="5">__The Battle of the Neighborhoods - Week 1__</font>

LEUNG CHUNG MING
---

<font size="3">__Introduction & Business Problem :__</font>

<font size="3">__Problem Background:__</font>

In this project, we will try to find the best location for lunch. Specifically, this report will target stakeholders interested in opening various international cuisine restaurants, cafes, food shops and entertainment. spot near the Kowloon in Hong Kong.

Due to the many various international cuisine restaurants, cafes, food shops and entertainment spots and eating spots near the Kowloon, we will try to detect locations that are not yet packed with various international cuisine restaurants, cafes, food shops and entertainment. We are also particularly interested in areas where there are no other restaurants nearby. Assuming that the first two conditions are met, we also want the position to be as close as possible to the Kowloon.

We will use our data science power to generate some of the most promising communities based on this standard. Then, the advantages of each area will be clearly expressed so that stakeholders can choose the best final position.

<font size="3">__Data Description:__</font>

According to the definition of our problem, the factors that influence our decision-making are:

Number of existing restaurants, cafes, food shops and entertainment. spots nearby
The number and distance of breakfast spots nearby, if there is any distance from the Kowloon. We decided to define our community with a circular grid of regularly spaced, centered around the center of Richmond.

The following data sources will be required to extract/generate the required information:

The center of the candidate area will be generated by an algorithm and will use the Foursquare to reverse geocode the number of breakfast points to obtain the approximate addresses of the centers of these areas, and will use Foursquare to get their type and location in each neighborhood will be used The center received the Foursquare API in the famous Kowloon, Hong Kong.

<font size="3">__Neighborhood  Candidates:__</font>

- Let’s create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will create a grid of cells covering our area of interest which is aprox. 1.6 killometers centered around Kowloon, Hong Kong.

- Let’s first find the latitude & longitude of Kowloon, using specific, well known address and Foursquare API.

In [63]:
# A description of the problem and a discussion of the background. (15 marks)

In [64]:
import folium


m = folium.Map(location=[22.3186, 114.1796])

In [65]:
m

In [66]:
m.save('index.html')

In [67]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

address = 'Kowloon, Hong Kong'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Singapore home are {}, {}.'.format(latitude, longitude))

usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


Libraries imported.
The geograpical coordinate of Singapore home are 22.3272706, 114.1898678.


In [68]:
neighborhood_latitude=22.3272706
neighborhood_longitude=114.1898678

In [69]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=HRMBKZUASN1NWO005IQK4TGG15UVEY5GCLJCYXHXW0VDP00K&client_secret=JSXFO23NR2OMICQSZRFQYDAZG1GMNRALXXACAFVNF5CGAM4C&v=20180604&ll=22.3272706,114.1898678&radius=500&limit=100'

In [70]:
results = requests.get(url).json()
#results

In [71]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [72]:
venues = results['response']['groups'][0]['items']
    
SGnearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]

# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)

# clean columns
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.head(10)

,name,categories,lat,lng
0,Lazy Creatures (慵懶生物),Café,22.327093,114.188935
1,Islam Food (清真牛肉館),Halal Restaurant,22.328211,114.190391
2,Kowloon City Market (九龍城街市),Market,22.329359,114.189015
3,Tei Mou Koon Dessert (地茂館甜品),Dessert Shop,22.328868,114.187629
4,Kung Wo Dou Bun Chong (公和荳品廠),Dessert Shop,22.329728,114.187646
5,Hop Shing Dessert (合成糖水),Dessert Shop,22.328387,114.190197
6,Woo Cow Hotpot (禾牛薈火鍋館),Hotpot Restaurant,22.329366,114.189867
7,Cheong Fat Thai Noodles (昌發泰國粉麵屋),Thai Restaurant,22.329231,114.190968
8,Islam Food (清真牛肉館),Halal Restaurant,22.329196,114.191516
9,Chaophraya 2 (昭拍耶),Thai Restaurant,22.328735,114.189851


In [73]:
# create map of Singapore place  using latitude and longitude values
map_sg = folium.Map(location=[latitude, longitude], zoom_start=20)

# add markers to map
for lat, lng, label in zip(SGnearby_venues['lat'], SGnearby_venues['lng'], SGnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.7,
    ).add_to(map_sg)  
    
map_sg

In [74]:
# A description of the data and how it will be used to solve the problem. (15 marks)

<font size="3">__Analysis__</font>

TBC

<font size="3">__Observations & Recommendations__</font>

TBC

<font size="3">__Conclusion__</font>

TBC